In [1]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi

# 使用 Qwen 模型
qwen_llm = ChatTongyi(
    model="qwen-max",              # Qwen 模型名称，例如 "qwen-7b" 或 "qwen-14b"
    api_key="sk-a31a563bce7146bf8c8ab93ad5fea537"   # 替换为实际的 API 密钥
)

# 调用 Qwen 模型
response = qwen_llm.invoke("你好！能介绍一下量子计算吗？")
print(response)

content='你好！当然可以。量子计算是一种基于量子力学原理的新型计算方式，它与传统的基于二进制的经典计算有着根本的不同。在经典计算机中，信息的基本单位是比特（bit），它可以表示0或1两种状态之一。而在量子计算中，信息的基本单位叫做量子比特（qubit）。量子比特利用了量子力学中的叠加态和纠缠等特性，使得一个量子比特不仅可以表示0或1，还可以同时处于这两种状态的任意线性组合之中。这种性质让量子计算机在处理某些特定问题时比传统计算机更加高效。\n\n### 量子计算的关键概念\n\n- **叠加**：这是指量子系统能够同时存在于多个状态之中。对于一个量子比特来说，这意味着它可以同时代表0和1。\n- **纠缠**：当两个或更多的量子比特以一种特殊的方式相互作用后，它们之间就会形成纠缠关系。即使相隔很远，对其中一个量子比特的操作也会立即影响到另一个量子比特的状态。\n- **干涉**：通过精确地控制量子比特之间的相互作用，可以使不同的路径之间发生建设性或者破坏性的干涉，从而提高正确答案的概率而减少错误答案出现的可能性。\n\n### 应用领域\n\n量子计算因其独特的性质，在解决某些类型的问题上具有潜在优势，比如：\n- 大整数分解：这对于当前广泛使用的加密算法构成了挑战。\n- 模拟量子系统：有助于新材料、药物的设计等领域。\n- 优化问题：如物流规划、金融建模等复杂优化场景。\n\n### 当前状态与发展\n\n尽管量子计算理论已经相当成熟，并且实验室里也成功构建了一些小型量子计算机原型机，但要实现大规模实用化的量子计算机仍然面临许多技术难题。包括但不限于如何长时间保持量子态的稳定性（即克服退相干问题）、如何有效地扩展量子比特数量以及开发适合量子硬件运行的新算法等等。\n\n总之，量子计算是一个非常前沿且快速发展的领域，虽然距离真正普及还有一定距离，但它展现出了解决未来一些重大科学和技术挑战的巨大潜力。' additional_kwargs={} response_metadata={'model_name': 'qwen-max', 'finish_reason': 'stop', 'request_id': '4051fb2a-069d-9f2c-9075-dc6702678a06', 'token_usage': {'input_tokens': 16,

In [8]:
from langchain_community.llms import Tongyi

import getpass 
import os 

# def _set_env(var: str): 
#     if not os.environ.get(var): 
#         os.environ[var] = getpass.getpass(f"{var}: ") 

# # api key: sk-a31a563bce7146bf8c8ab93ad5fea537
# _set_env("DASHSCOPE_API_KEY")

# llm = Tongyi()

def chatbot(state: State):
    return {"messages": [qwen_llm.invoke(state["messages"])]}

In [9]:
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [31]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

def invoke_graph_(user_input: str):
    answers = graph.invoke({"messages": [("user", user_input)]})
    print(answers)

while True:
    # try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        # stream_graph_updates(user_input)
        invoke_graph_(user_input)
    
    # except:
    #     # fallback if input() is not available
    #     user_input = "What do you know about LangGraph?"
    #     print("User: " + user_input)
    #     stream_graph_updates(user_input)
    #     break

User:  q


Goodbye!


In [21]:
class StateTest(TypedDict):
    messages: Annotated[list, add_messages]

stateTest: StateTest = {
    'messages': ['123']
}

stateTest['messages'] = ['456']

stateTest['messages']

['456']

In [6]:
from langchain_core.runnables import RunnableConfig
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

def reducer(a: list, b: int | None) -> list:
    if b is not None:
        return a + [b]
    return a

class State(TypedDict):
    x: Annotated[list, reducer]

class ConfigSchema(TypedDict):
    r: float

graph = StateGraph(State, config_schema=ConfigSchema)

def node(state: State, config: RunnableConfig) -> dict:
    r = config["configurable"].get("r", 1.0)
    x = state["x"][-1]
    next_value = x * r * (1 - x)
    return {"x": next_value}

graph.add_node("A", node)
graph.set_entry_point("A")
graph.set_finish_point("A")
compiled = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}


step1 = compiled.invoke({"x": 0.5}, {"configurable": {"r": 3.0, "thread_id": "1"}})
print(step1)
{'x': [0.5, 0.75]}

{'x': [0.5, 0.75]}


{'x': [0.5, 0.75]}

In [8]:
step1 = compiled.invoke({"x": 0.6}, {"configurable": {"r": 3.0, "thread_id": "1"}})
print(step1)

{'x': [0.5, 0.75, 0.6, 0.72]}


In [1]:
if messages := []:
    print("[]")
else:
    print("None")

None


In [6]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.pydantic_v1 import BaseModel, Field

class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

class GetPrice(BaseModel):
    '''Get the price of a specific product.'''

    product: str = Field(..., description="The product to look up.")


# 使用 Qwen 模型
llm = ChatTongyi(
    model="qwen-max",              # Qwen 模型名称，例如 "qwen-7b" 或 "qwen-14b"
    api_key="sk-a31a563bce7146bf8c8ab93ad5fea537"   # 替换为实际的 API 密钥
)

llm_with_tools = llm.bind_tools([GetWeather, GetPrice])
llm_with_tools.invoke("what is the weather like in San Francisco",)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'GetMoney', 'arguments': '{"Money": "San Francisco, CA"}'}, 'index': 0, 'id': 'call_65338a678da545f3a03ae5', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '76eca852-ad9d-9773-b55b-d139d8cceb6c', 'token_usage': {'input_tokens': 252, 'output_tokens': 20, 'total_tokens': 272}}, id='run-21383236-5056-4a55-a1f0-586de827fb29-0', tool_calls=[{'name': 'GetMoney', 'args': {'Money': 'San Francisco, CA'}, 'id': 'call_65338a678da545f3a03ae5', 'type': 'tool_call'}])